In [15]:
import pandas as pd
import os

# 打开 Excel 文件
excel_file_path = "data/3-读取excel.xlsx"  # Excel 文件路径

if os.path.exists(excel_file_path):
    with pd.ExcelFile(excel_file_path) as xls:
        # 输出所有有内容的工作表名称
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            if not df.empty:
                print(f"Sheet name with content: {sheet_name}")
else:
    print("找不到指定的 Excel 文件。")

Sheet name with content: 工作表1


In [16]:
def export_excel_to_lua(excel_path, sheet_name, lua_output_path):
    # 读取 Excel 文件
    df = pd.read_excel(excel_path, sheet_name=sheet_name)

    # 找到键名所在的行（第一个非注释行）
    key_row_index = None
    header_comment = []
    for idx, row in df.iterrows():
        if not pd.isna(row[0]) and str(row[0]).startswith('#'):
            header_comment.append(str(row[0]))  # 保存注释行
        elif not pd.isna(row[0]) and not str(row[0]).startswith('#'):
            key_row_index = idx
            break

    if key_row_index is None:
        print(f"未找到有效的键名行，跳过工作表 {sheet_name}")
        return

    # 获取键名
    keys = df.iloc[key_row_index]
    keys_to_export = []
    for column_name in df.columns:
        key_name = keys[column_name]
        if not pd.isna(key_name) and not str(key_name).startswith('#'):
            keys_to_export.append((column_name, key_name))

    # 打开或创建 Lua 输出文件
    with open(lua_output_path, 'w', encoding='utf-8') as lua_file:
        for comment in header_comment:
            lua_file.write(f'-- {comment}\n')
        lua_file.write('local config = {\n')

        # 遍历 Excel 每一行，从键名行之后开始
        skip_cnt = 0
        for index, row in df.iloc[key_row_index + 1:].iterrows():
            if pd.isna(row[0]) or str(row[0]).startswith('#'):  # 如果第一列为空，跳过该行
                skip_cnt += 1
                continue

            lua_file.write(f'    [{index - key_row_index - skip_cnt}] = {{\n')
            for column_name, key in keys_to_export:
                value = row[column_name]
                # 检查数据类型并格式化为 Lua 格式
                if isinstance(value, str):
                    # 如果值是以 "|" 分隔的字符串，拆分为数组
                    if '|' in value:
                        split_values = value.split('|')
                        lua_file.write(f'        ["{key}"] = {{\n')
                        for i, split_value in enumerate(split_values, start=1):
                            lua_file.write(f'            [{i}] = {split_value},\n')
                        lua_file.write('        },\n')
                    else:
                        lua_value = f'"{value}"'
                        lua_file.write(f'        ["{key}"] = {lua_value},\n')
                elif pd.isna(value):
                    # 跳过空值
                    continue
                else:
                    lua_value = str(value)
                    lua_file.write(f'        ["{key}"] = {lua_value},\n')

            lua_file.write('    },\n')

        lua_file.write('}\n\nreturn config\n')

In [17]:
sheet_name = '工作表1'
lua_file_path = f"output/{sheet_name}_config.lua"  # 导出 Lua 文件路径
export_excel_to_lua(excel_file_path, sheet_name, lua_file_path)
print(f"配置文件成功导出到 {lua_file_path}")

配置文件成功导出到 output/工作表1_config.lua


C:\Users\9you\AppData\Local\Temp\ipykernel_166160\294892096.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if not pd.isna(row[0]) and str(row[0]).startswith('#'):
C:\Users\9you\AppData\Local\Temp\ipykernel_166160\294892096.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif not pd.isna(row[0]) and not str(row[0]).startswith('#'):
C:\Users\9you\AppData\Local\Temp\ipykernel_166160\294892096.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, us